In [5]:
from IPython.display import HTML
HTML('''
    <style> body {font-family: "Roboto Condensed Light", "Roboto Condensed";} h2 {padding: 10px 12px; background-color: #E64626; position: static; color: #ffffff; font-size: 40px;} .text_cell_render p { font-size: 15px; } .text_cell_render h1 { font-size: 30px; } h1 {padding: 10px 12px; background-color: #E64626; color: #ffffff; font-size: 40px;} .text_cell_render h3 { padding: 10px 12px; background-color: #0148A4; position: static; color: #ffffff; font-size: 20px;} h4:before{ 
    content: "@"; font-family:"Wingdings"; font-style:regular; margin-right: 4px;} .text_cell_render h4 {padding: 8px; font-family: "Roboto Condensed Light"; position: static; font-style: italic; background-color: #FFB800; color: #ffffff; font-size: 18px; text-align: center; border-radius: 5px;}input[type=submit] {background-color: #E64626; border: solid; border-color: #734036; color: white; padding: 8px 16px; text-decoration: none; margin: 4px 2px; cursor: pointer; border-radius: 20px;}</style>
''')

# 👥 Group Information

<p><strong>Code Language:</strong> <span style="font-size:18px;">Python</span></p>

<table style="font-size:18px;">
  <tr>
    <th>Name</th>
    <th>SID</th>
  </tr>
  <tr>
    <td>Ngoc Minh Dao</td>
    <td>520577590</td>
  </tr>
  <tr>
    <td>Manh Duc Nguyen</td>
    <td>520561337</td>
  </tr>
</table>

# Sydney Public Transport Analysis

### Import necessary libraries

In [106]:
# Imports for spatial data
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import time
import numpy as np
from shapely.geometry import Point, Polygon, MultiPolygon
from geoalchemy2 import Geometry, WKTElement

# Imports for pgadmin
from sqlalchemy import create_engine, text
import psycopg2
import psycopg2.extras
import json
from sqlalchemy import text

### Connect to pgAdmin

In [108]:
credentials = "Credentials.json"

def pgconnect(credential_filepath, db_schema="public"):
    with open(credential_filepath) as f:
        db_conn_dict = json.load(f)
        host       = db_conn_dict['host']
        db_user    = db_conn_dict['user']
        db_pw      = db_conn_dict['password']
        default_db = db_conn_dict['user']
        port       = db_conn_dict['port']
        try:
            db = create_engine(f'postgresql+psycopg2://{db_user}:{db_pw}@{host}:{port}/{default_db}', echo=False)
            conn = db.connect()
            print('Connected successfully.')
        except Exception as e:
            print("Unable to connect to the database.")
            print(e)
            db, conn = None, None
        return db,conn

def query(conn, sqlcmd, args=None, df=True):
    result = pd.DataFrame() if df else None
    try:
        if df:
            result = pd.read_sql_query(sqlcmd, conn, params=args)
        else:
            result = conn.execute(text(sqlcmd), args).fetchall()
            result = result[0] if len(result) == 1 else result
    except Exception as e:
        print("Error encountered: ", e, sep='\n')
    return result

In [110]:
db, conn = pgconnect(credentials)

Connected successfully.


### SRID Setup

In [112]:
srid = 4326

In [114]:
def create_wkt_element(geom, srid):
    if geom.geom_type == 'Polygon':
        geom = MultiPolygon([geom])
    return WKTElement(geom.wkt, srid)

## Task 1: Import and Clean datasets

### 1.1 Load + clean Sydney Trains data

#### 1.1.1 Load data

In [122]:
# Load the file
trains = gpd.read_file("SydneyTrainRoutes/sydneytrains/SydneyTrains.shp")
print(trains.columns)
# trains.head(5)

Index(['objectid', 'shape_id', 'route_id', 'agency_id', 'route_shor',
       'route_long', 'route_desc', 'route_type', 'route_colo', 'route_text',
       'exact_time', 'route_ty00', 'st_length(', 'geometry'],
      dtype='object')


,objectid,shape_id,route_id,agency_id,route_shor,route_long,route_desc,route_type,route_colo,route_text,exact_time,route_ty00,st_length(,geometry
0,1,APS_1a,APS_1a,SydneyTrains,T8,City Circle to Macarthur via Airport,T8 Airport & South Line,Rail,00954C,FFFFFF,None,Rail,67399.109358,"LINESTRING (1.68e+07 -4.01e+06, 1.68e+07 -4.01..."
1,54,SCO_1a,SCO_1a,NSWTrains,SCO,Bondi Junction and Central to Bomaderry,South Coast Line,Rail,005AA3,FFFFFF,None,Rail,194277.573535,"LINESTRING (1.68e+07 -4.01e+06, 1.68e+07 -4.01..."
2,55,SCO_1b,SCO_1b,NSWTrains,SCO,Bondi Junction and Central to Port Kembla,South Coast Line,Rail,005AA3,FFFFFF,None,Rail,118023.250081,"LINESTRING (1.68e+07 -4.01e+06, 1.68e+07 -4.01..."
3,56,SCO_2a,SCO_2a,NSWTrains,SCO,Bomaderry to Central and Bondi Junction,South Coast Line,Rail,005AA3,FFFFFF,None,Rail,194277.573535,"LINESTRING (1.68e+07 -4.14e+06, 1.68e+07 -4.14..."
4,57,SCO_2b,SCO_2b,NSWTrains,SCO,Port Kembla to Central and Bondi Junction,South Coast Line,Rail,005AA3,FFFFFF,None,Rail,118023.250081,"LINESTRING (1.68e+07 -4.09e+06, 1.68e+07 -4.09..."


In [145]:
# Find all the Sydney trains
sydney_trains = trains[trains["agency_id"].str.startswith("Sydney")]
# sydney_trains.head(5)

,objectid,shape_id,route_id,agency_id,route_shor,route_long,route_desc,route_type,route_colo,route_text,exact_time,route_ty00,st_length(,geometry
0,1,APS_1a,APS_1a,SydneyTrains,T8,City Circle to Macarthur via Airport,T8 Airport & South Line,Rail,00954C,FFFFFF,None,Rail,67399.109358,"LINESTRING (1.68e+07 -4.01e+06, 1.68e+07 -4.01..."
9,2,APS_1c,APS_1c,SydneyTrains,T8,City Circle to Macarthur via Sydenham,T8 Airport & South Line,Rail,00954C,FFFFFF,None,Rail,64669.017674,"LINESTRING (1.68e+07 -4.01e+06, 1.68e+07 -4.01..."
10,3,APS_1e,APS_1e,SydneyTrains,T8,City Circle to Leppington via Sydenham,T8 Airport & South Line,Rail,00954C,FFFFFF,None,Rail,57993.335454,"LINESTRING (1.68e+07 -4.01e+06, 1.68e+07 -4.01..."
11,4,APS_1f,APS_1f,SydneyTrains,T8,City Circle to Leppington via Sydenham,T8 Airport & South Line,Rail,00954C,FFFFFF,None,Rail,57991.813538,"LINESTRING (1.68e+07 -4.01e+06, 1.68e+07 -4.01..."
12,7,APS_2d,APS_2d,SydneyTrains,T8,Macarthur to City Circle via Sydenham,T8 Airport & South Line,Rail,00954C,FFFFFF,None,Rail,64667.495758,"LINESTRING (1.68e+07 -4.04e+06, 1.68e+07 -4.04..."


In [140]:
# Check data types
sydney_trains.dtypes

objectid         int64
shape_id        object
route_id        object
agency_id       object
route_shor      object
route_long      object
route_desc      object
route_type      object
route_colo      object
route_text      object
exact_time      object
route_ty00      object
st_length(     float64
geometry      geometry
dtype: object

#### 1.1.2 SRID Transformation

In [147]:
sydney_trains = sydney_trains.copy()  # creating a copy of the original for later
sydney_trains['geom'] = sydney_trains['geometry'].apply(lambda x: create_wkt_element(geom=x,srid=srid))  # applying the function
sydney_trains = sydney_trains.drop(columns="geometry")  # deleting the old copy
# sydney_trains.head(5)

,objectid,shape_id,route_id,agency_id,route_shor,route_long,route_desc,route_type,route_colo,route_text,exact_time,route_ty00,st_length(,geom
0,1,APS_1a,APS_1a,SydneyTrains,T8,City Circle to Macarthur via Airport,T8 Airport & South Line,Rail,00954C,FFFFFF,None,Rail,67399.109358,"LINESTRING (16832327.3866 -4013201.665800001, ..."
9,2,APS_1c,APS_1c,SydneyTrains,T8,City Circle to Macarthur via Sydenham,T8 Airport & South Line,Rail,00954C,FFFFFF,None,Rail,64669.017674,"LINESTRING (16832327.3866 -4013201.665800001, ..."
10,3,APS_1e,APS_1e,SydneyTrains,T8,City Circle to Leppington via Sydenham,T8 Airport & South Line,Rail,00954C,FFFFFF,None,Rail,57993.335454,"LINESTRING (16832327.3866 -4013201.665800001, ..."
11,4,APS_1f,APS_1f,SydneyTrains,T8,City Circle to Leppington via Sydenham,T8 Airport & South Line,Rail,00954C,FFFFFF,None,Rail,57991.813538,"LINESTRING (16832330.7499 -4013201.9989, 16832..."
12,7,APS_2d,APS_2d,SydneyTrains,T8,Macarthur to City Circle via Sydenham,T8 Airport & South Line,Rail,00954C,FFFFFF,None,Rail,64667.495758,"LINESTRING (16786653.3795 -4038476.7914, 16787..."


In [130]:
# Check for missing values 
null_val = sydney_trains.isnull().sum()
print(f"Missing values per column:\n{null_val}")

Missing values per column:
objectid       0
shape_id       0
route_id       0
agency_id      0
route_shor     0
route_long     0
route_desc     0
route_type     0
route_colo     0
route_text     0
exact_time    54
route_ty00     0
st_length(     0
geometry       0
dtype: int64


In [153]:
# For "exact_time", fill with "Unknown"
sydney_trains['exact_time'] = sydney_trains['exact_time'].fillna('Unknown')
# sydney_trains.head(5)

#### 1.1.3 Create table for query

In [ ]:
conn.execute(text("""
DROP TABLE IF EXISTS sydney_trains;
CREATE TABLE sydney_trains (
    objectid     INTEGER PRIMARY KEY,
    shape_id     VARCHAR(255),
    route_id     VARCHAR(255),
    agency_id    VARCHAR(255),
    route_shor   VARCHAR(255),
    route_long   VARCHAR(255),
    route_desc   VARCHAR(255),
    route_type   VARCHAR(255),
    route_colo   VARCHAR(255),
    route_text   VARCHAR(255),
    exact_time   VARCHAR(255),
    route_ty00   VARCHAR(255),
    st_length    DOUBLE PRECISION,
    geom         GEOMETRY(LINESTRING, 4326)
);
"""))

### 1.2 Load + clean Train Station Entrance Locations data

In [61]:
# Load the file
entrance_loc = pd.read_csv("TrainStationEntranceLocations/stationentrances2020_v4.csv")
entrance_loc.head(5)

,Train_Station,Street_Name,Street_Type,Entrance_Type,LAT,LONG,Exit_Number
0,Aberdeen,Macqueen,St,Ramp,-32.166886,150.891957,NaN
1,Aberdeen,Macqueen,St,Stairs,-32.166900,150.891975,NaN
2,Adamstown,Park,Ave,Path,-32.933706,151.720452,NaN
3,Adamstown,Park,Ave,Path,-32.933827,151.720236,NaN
4,Adamstown,St James,Rd,Stairs,-32.933414,151.720363,NaN


In [30]:
# Check data types
entrance_loc.dtypes

Train_Station     object
Street_Name       object
Street_Type       object
Entrance_Type     object
LAT              float64
LONG             float64
Exit_Number      float64
dtype: object

In [32]:
# Check for missing values 
null_val = entrance_loc.isnull().sum()
print(f"Missing values per column:\n{null_val}")

Missing values per column:
Train_Station       0
Street_Name         0
Street_Type        28
Entrance_Type       0
LAT                 0
LONG                0
Exit_Number      1024
dtype: int64


In [63]:
# For Street_Type, fill with "Unknown"
entrance_loc['Street_Type'] = entrance_loc['Street_Type'].fillna('Unknown')
# For Exit_Number, assuming missing = 0 exits
entrance_loc['Exit_Number'] = entrance_loc['Exit_Number'].fillna(0).astype(int)
entrance_loc.head(5)

,Train_Station,Street_Name,Street_Type,Entrance_Type,LAT,LONG,Exit_Number
0,Aberdeen,Macqueen,St,Ramp,-32.166886,150.891957,0
1,Aberdeen,Macqueen,St,Stairs,-32.166900,150.891975,0
2,Adamstown,Park,Ave,Path,-32.933706,151.720452,0
3,Adamstown,Park,Ave,Path,-32.933827,151.720236,0
4,Adamstown,St James,Rd,Stairs,-32.933414,151.720363,0


In [84]:
entrance_loc['geom'] = gpd.points_from_xy(entrance_loc.LONG, entrance_loc.LAT)      # creating the geometry column
entrance_loc = entrance_loc.drop(columns = ['LONG', 'LAT'])        # removing the old latitude/longitude fields
entrance_loc.head(5)

,Train_Station,Street_Name,Street_Type,Entrance_Type,Exit_Number,geom
0,Aberdeen,Macqueen,St,Ramp,0,POINT (150.89 -32.167)
1,Aberdeen,Macqueen,St,Stairs,0,POINT (150.89 -32.167)
2,Adamstown,Park,Ave,Path,0,POINT (151.72 -32.934)
3,Adamstown,Park,Ave,Path,0,POINT (151.72 -32.934)
4,Adamstown,St James,Rd,Stairs,0,POINT (151.72 -32.933)


### 1.3 Load + clean Train Station Entries Exits data

In [65]:
# Load the file
entries_exit = pd.read_csv("TrainStationEntriesExits/train-station-entries-exits-data-may-2025.csv")
entries_exit.head(5)

,MonthYear,Station,Station_Type,Entry_Exit,Trip
0,Aug-24,Aberdeen Station,train,Entry,Less than 50
1,Aug-24,Aberdeen Station,train,Exit,Less than 50
2,Aug-24,Adamstown Station,train,Entry,2585
3,Aug-24,Adamstown Station,train,Exit,2391
4,Aug-24,Albion Park Station,train,Entry,6919


In [86]:
# Check data types
entries_exit.dtypes

MonthYear        object
Station          object
Station_Type     object
Entry_Exit       object
Trip             object
dtype: object

In [94]:
# # Convert MonthYear to datetime
# entries_exit['MonthYear'] = pd.to_datetime(entries_exit['MonthYear'], 
#                                        format='%Y-%m',     # e.g. '2024-08'
#                                        errors='coerce')
# entries_exit.dtypes

MonthYear        datetime64[ns]
Station                  object
Station_Type             object
Entry_Exit               object
Trip                     object
dtype: object

In [100]:
# Check for missing values 
null_val = entries_exit.isnull().sum()
print(f"Missing values per column:\n{null_val}")

Missing values per column:
MonthYear        6282
Station             0
Station_Type        0
Entry_Exit          0
Trip                0
dtype: int64


### 1.4 Load + clean Opal Patronage data